In [1]:
!pwd
import sys

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124\run_objective_rs


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [3]:
# tensorflow2.0 + kerasでGPUメモリの使用量を抑える方法(最小限だけ使うように設定)
import os

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [5]:
import sys

In [6]:
sys.path.append(
    r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLP_for_ensemble\20201124"
)
import moa_MLPs_funcs

Tensorflow version 2.3.1
n_classes: 206


In [7]:
import optuna


def objective(trial):
    model_type = "rs"
    params = {}

    activations = []
    for i in range(8):
        activation = trial.suggest_categorical(f"activation{i}", ["relu", "elu", "selu"])
        activations.append(activation)
    params["activations"] = activations

    denses = []
    for i in range(6):
        #dense = trial.suggest_int(f"dense{i}", 128, 1024)
        dense = trial.suggest_int(f"dense{i}", 256, 1024)  # oldの結果より
        denses.append(dense)
    params["denses"] = denses

    drop_rates = []
    for i in range(2):
        #drop_rate = trial.suggest_uniform(f"drop_rate{i}", 0.1, 0.9)
        drop_rate = trial.suggest_uniform(f"drop_rate{i}", 0.5, 0.9)  # oldの結果より
        drop_rates.append(drop_rate)
    params["drop_rates"] = drop_rates

    print("-" * 100)
    print(f"params: {params}")

    oof_score, Y_pred = moa_MLPs_funcs.train_and_evaluate(
        model_type=model_type, params=params
    )

    return oof_score

In [8]:
%%time

import warnings
warnings.filterwarnings('ignore')

#n_trials = 400
n_trials = 100
#n_trials = 3

study = optuna.create_study(
    study_name="study",
    storage=f"sqlite:///study.db",
    load_if_exists=True,
    direction="minimize",
    sampler=optuna.samplers.TPESampler(seed=1),
)
study.optimize(objective, n_trials=n_trials)
study.trials_dataframe().to_csv(f"objective_history.csv", index=False)
with open(f"objective_best_params.txt", mode="w") as f:
    f.write(str(study.best_params))
print(f"\nstudy.best_params:\n{study.best_params}")
print(f"\nstudy.best_value:\n{study.best_value}")

[I 2020-11-26 17:21:25,123] A new study created in RDB with name: study


----------------------------------------------------------------------------------------------------
params: {'activations': ['elu', 'relu', 'relu', 'elu', 'elu', 'relu', 'relu', 'elu'], 'denses': [839, 1005, 764, 646, 537, 434], 'drop_rates': [0.8385243666744069, 0.62530940677291]}

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer

Epoch 1/50
19/19 - 2s - loss: 0.7342 - binary_crossentropy: 0.7341 - val_loss: 2.3812 - val_binary_crossentropy: 2.3808
Epoch 2/50
19/19 - 1s - loss: 0.4104 - binary_crossentropy: 0.4098 - val_loss: 0.3204 - val_binary_crossentropy

Epoch 6/50
19/19 - 1s - loss: 0.0233 - binary_crossentropy: 0.0187 - val_loss: 0.0259 - val_binary_crossentropy: 0.0212
Epoch 7/50
19/19 - 1s - loss: 0.0229 - binary_crossentropy: 0.0184 - val_loss: 0.0239 - val_binary_crossentropy: 0.0193
Epoch 8/50
19/19 - 1s - loss: 0.0226 - binary_crossentropy: 0.0181 - val_loss: 0.0230 - val_binary_crossentropy: 0.0185
Epoch 9/50
19/19 - 1s - loss: 0.0223 - binary_crossentropy: 0.0178 - val_loss: 0.0224 - val_binary_crossentropy: 0.0179
Epoch 10/50
19/19 - 1s - loss: 0.0221 - binary_crossentropy: 0.0176 - val_loss: 0.0222 - val_binary_crossentropy: 0.0176
Epoch 11/50
19/19 - 1s - loss: 0.0220 - binary_crossentropy: 0.0174 - val_loss: 0.0219 - val_binary_crossentropy: 0.0173
Epoch 12/50
19/19 - 1s - loss: 0.0219 - binary_crossentropy: 0.0173 - val_loss: 0.0218 - val_binary_crossentropy: 0.0171
Epoch 13/50
19/19 - 1s - loss: 0.0217 - binary_crossentropy: 0.0171 - val_loss: 0.0217 - val_binary_crossentropy: 0.0171
Epoch 14/50
19/19 - 1s - loss: 0.021

Epoch 20/50
19/19 - 1s - loss: 0.0210 - binary_crossentropy: 0.0163 - val_loss: 0.0216 - val_binary_crossentropy: 0.0170
Epoch 21/50
19/19 - 1s - loss: 0.0209 - binary_crossentropy: 0.0162 - val_loss: 0.0215 - val_binary_crossentropy: 0.0169
Epoch 22/50
19/19 - 1s - loss: 0.0209 - binary_crossentropy: 0.0162 - val_loss: 0.0215 - val_binary_crossentropy: 0.0169
Epoch 23/50
19/19 - 1s - loss: 0.0207 - binary_crossentropy: 0.0161 - val_loss: 0.0215 - val_binary_crossentropy: 0.0170
Epoch 24/50
19/19 - 1s - loss: 0.0206 - binary_crossentropy: 0.0160 - val_loss: 0.0214 - val_binary_crossentropy: 0.0167
Epoch 25/50
19/19 - 1s - loss: 0.0205 - binary_crossentropy: 0.0159 - val_loss: 0.0214 - val_binary_crossentropy: 0.0168
Epoch 26/50
19/19 - 1s - loss: 0.0205 - binary_crossentropy: 0.0158 - val_loss: 0.0216 - val_binary_crossentropy: 0.0170
Epoch 27/50
19/19 - 1s - loss: 0.0205 - binary_crossentropy: 0.0158 - val_loss: 0.0214 - val_binary_crossentropy: 0.0168
Epoch 28/50
19/19 - 1s - loss: 0

Epoch 34/50
19/19 - 1s - loss: 0.0199 - binary_crossentropy: 0.0151 - val_loss: 0.0213 - val_binary_crossentropy: 0.0166
Epoch 35/50
19/19 - 1s - loss: 0.0198 - binary_crossentropy: 0.0151 - val_loss: 0.0212 - val_binary_crossentropy: 0.0165
Epoch 36/50
19/19 - 1s - loss: 0.0198 - binary_crossentropy: 0.0151 - val_loss: 0.0213 - val_binary_crossentropy: 0.0166
Epoch 37/50
19/19 - 1s - loss: 0.0198 - binary_crossentropy: 0.0150 - val_loss: 0.0214 - val_binary_crossentropy: 0.0167
Epoch 38/50
19/19 - 1s - loss: 0.0196 - binary_crossentropy: 0.0148 - val_loss: 0.0212 - val_binary_crossentropy: 0.0165
Epoch 39/50
19/19 - 1s - loss: 0.0194 - binary_crossentropy: 0.0146 - val_loss: 0.0212 - val_binary_crossentropy: 0.0165
Epoch 40/50
19/19 - 1s - loss: 0.0193 - binary_crossentropy: 0.0145 - val_loss: 0.0212 - val_binary_crossentropy: 0.0165
Epoch 41/50
19/19 - 1s - loss: 0.0193 - binary_crossentropy: 0.0145 - val_loss: 0.0211 - val_binary_crossentropy: 0.0164
Epoch 42/50
19/19 - 1s - loss: 0

...

Epoch 21/50
19/19 - 1s - loss: 0.0200 - binary_crossentropy: 0.0153 - val_loss: 0.0212 - val_binary_crossentropy: 0.0165
Epoch 22/50
19/19 - 1s - loss: 0.0199 - binary_crossentropy: 0.0151 - val_loss: 0.0214 - val_binary_crossentropy: 0.0167
Epoch 23/50
19/19 - 1s - loss: 0.0197 - binary_crossentropy: 0.0150 - val_loss: 0.0214 - val_binary_crossentropy: 0.0167
Epoch 24/50
19/19 - 1s - loss: 0.0197 - binary_crossentropy: 0.0149 - val_loss: 0.0213 - val_binary_crossentropy: 0.0165
Epoch 25/50
19/19 - 1s - loss: 0.0195 - binary_crossentropy: 0.0147 - val_loss: 0.0213 - val_binary_crossentropy: 0.0166
Epoch 26/50
19/19 - 1s - loss: 0.0194 - binary_crossentropy: 0.0146 - val_loss: 0.0216 - val_binary_crossentropy: 0.0168
Epoch 27/50
19/19 - 1s - loss: 0.0193 - binary_crossentropy: 0.0145 - val_loss: 0.0214 - val_binary_crossentropy: 0.0167
Epoch 28/50
19/19 - 1s - loss: 0.0191 - binary_crossentropy: 0.0143 - val_loss: 0.0214 - val_binary_crossentropy: 0.0166
Epoch 29/50
19/19 - 1s - loss: 0

Epoch 35/50
19/19 - 1s - loss: 0.0180 - binary_crossentropy: 0.0131 - val_loss: 0.0213 - val_binary_crossentropy: 0.0166
Epoch 36/50
19/19 - 1s - loss: 0.0179 - binary_crossentropy: 0.0130 - val_loss: 0.0213 - val_binary_crossentropy: 0.0165
Epoch 37/50
19/19 - 1s - loss: 0.0178 - binary_crossentropy: 0.0128 - val_loss: 0.0213 - val_binary_crossentropy: 0.0165
Epoch 38/50
19/19 - 1s - loss: 0.0177 - binary_crossentropy: 0.0127 - val_loss: 0.0214 - val_binary_crossentropy: 0.0166
Epoch 39/50
19/19 - 1s - loss: 0.0175 - binary_crossentropy: 0.0125 - val_loss: 0.0214 - val_binary_crossentropy: 0.0165
Epoch 40/50
19/19 - 1s - loss: 0.0174 - binary_crossentropy: 0.0124 - val_loss: 0.0215 - val_binary_crossentropy: 0.0166
Epoch 41/50
19/19 - 1s - loss: 0.0172 - binary_crossentropy: 0.0122 - val_loss: 0.0217 - val_binary_crossentropy: 0.0169
Epoch 42/50
19/19 - 1s - loss: 0.0171 - binary_crossentropy: 0.0121 - val_loss: 0.0218 - val_binary_crossentropy: 0.0169
Epoch 43/50
19/19 - 1s - loss: 0

Epoch 49/50
19/19 - 1s - loss: 0.0154 - binary_crossentropy: 0.0102 - val_loss: 0.0216 - val_binary_crossentropy: 0.0166
Epoch 50/50
19/19 - 1s - loss: 0.0152 - binary_crossentropy: 0.0100 - val_loss: 0.0217 - val_binary_crossentropy: 0.0166

oof_score: 0.016132312722320408


[I 2020-11-26 17:33:43,388] Trial 2 finished with value: 0.016132312722320408 and parameters: {'activation0': 'elu', 'activation1': 'selu', 'activation2': 'elu', 'activation3': 'elu', 'activation4': 'relu', 'activation5': 'relu', 'activation6': 'elu', 'activation7': 'relu', 'dense0': 984, 'dense1': 909, 'dense2': 883, 'dense3': 687, 'dense4': 889, 'dense5': 712, 'drop_rate0': 0.53933873353322, 'drop_rate1': 0.6684430500020209}. Best is trial 0 with value: 0.016127496881749984.


oof_auc_score: 0.6733431815196527
----------------------------------------------------------------------------------------------------

study.best_params:
{'activation0': 'elu', 'activation1': 'relu', 'activation2': 'relu', 'activation3': 'elu', 'activation4': 'elu', 'activation5': 'relu', 'activation6': 'relu', 'activation7': 'elu', 'dense0': 839, 'dense1': 1005, 'dense2': 764, 'dense3': 646, 'dense4': 537, 'dense5': 434, 'drop_rate0': 0.8385243666744069, 'drop_rate1': 0.62530940677291}

study.best_value:
0.016127496881749984
Wall time: 12min 18s
